In [1]:
#plots.py
import os
import pandas
import pandas_datareader.data as web
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms
import datetime
import math

def gather_data(data_codes, start, end = datetime.datetime.today(), freq = "A"):
    i = 0
    # dct.items() calls key and value that key points to
    for key, val in data_codes.items():
        if i == 0:
            # Create dataframe for first variable, then rename column
            df = web.DataReader(val, "fred", start, end).resample(freq).mean()
            df.rename(columns = {val:key}, inplace = True) 
            # setting i to None will cause the next block of code to execute,
            # placing data within df instead of creating a new dataframe for
            # each variable
            i = None
        else:
            # If dataframe already exists, add new column
            df[key] = web.DataReader(val, "fred", start, end).resample(freq).mean()

    return df

In [2]:
data_codes  = {#"IOER (%)":"IOER",
              "30 Y (%)": "DGS30",
              "EFFR (%)": "DFF",
              "1 M (%)": "DGS1MO",
              "3 M (%)": "DGS3MO",
              "6 M (%)": "DGS6MO",               
              "1 Y (%)": "DGS1",
              "2 Y (%)": "DGS2",
              "3 Y (%)": "DGS3",
              "5 Y (%)": "DGS5",
              "7 Y (%)": "DGS7",
              "10 Y (%)": "DGS10",
              "20 Y (%)": "DGS20",
              "RGDP":"GDPC1",
              "NGDP":"GDP",
              "5 Year TIPS":"DFII5",
              "10 Year TIPS": "DFII10",
             
}
               
# Select start and end dates
start = datetime.datetime(1983, 1, 1)
end = datetime.datetime.today()

# freq refers to data frequency. Choose "D", "W", "M", "Q", "A"
# a number may also be place in front of a letter. "2D" indicates
#       alternating days
data = gather_data(data_codes = data_codes, start = start, end = end, freq = "M")#.dropna()

In [3]:
# Interpolate Quarterly Data for Monthly comparisons
data["RGDP"] = data["RGDP"].interpolate(method='polynomial', order=3)
data["NGDP"] = data["NGDP"].interpolate(method='polynomial', order=3)

In [4]:
# Create Indicator Variables For Identifying Yield Curve Inversion and Recession
data["EFFR > 2Y"] = False
data["Recession"] = False
data["EFFR > 2Y"][data["2 Y (%)"] < data["EFFR (%)"]] = True
# Defnitions of Recession are somewhat controversial
#  Here I use a reasonable reformulation of the 2 consecutive quarters of real gdp contraction 
# If GDP is lower than it was 6 months ago (i.e., 2 quarters ago), indicate a recession
data["Recession"][data["RGDP"]<data["RGDP"].shift(6)] = True
data[data["Recession"]]

C:\Users\JLCat\AppData\Local\Temp\ipykernel_1468\491588744.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["EFFR > 2Y"][data["2 Y (%)"] < data["EFFR (%)"]] = True
C:\Users\JLCat\AppData\Local\Temp\ipykernel_1468\491588744.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Recession"][data["RGDP"]<data["RGDP"].shift(6)] = True


,30 Y (%),EFFR (%),1 M (%),3 M (%),6 M (%),1 Y (%),2 Y (%),3 Y (%),5 Y (%),7 Y (%),10 Y (%),20 Y (%),RGDP,NGDP,5 Year TIPS,10 Year TIPS,EFFR > 2Y,Recession
DATE,,,,,,,,,,,,,,,,,,
1990-09-30,9.028947,8.195000,NaN,7.597895,7.698421,7.763158,8.079474,8.267368,8.513684,8.786842,8.893158,NaN,9352.215816,6009.761981,NaN,NaN,True,True
1990-10-31,8.857727,8.112581,NaN,7.400000,7.529545,7.551364,7.877727,8.068636,8.327727,8.593636,8.719545,NaN,9318.876000,6004.733000,NaN,NaN,True,True
1990-11-30,8.540500,7.806667,NaN,7.290500,7.385000,7.313500,7.599500,7.737000,8.022500,8.276500,8.392000,NaN,9292.169392,6006.161585,NaN,NaN,True,True
1990-12-31,8.237000,7.312903,NaN,6.949500,7.026500,7.050500,7.314000,7.466000,7.726500,8.000500,8.075000,NaN,9275.661751,6015.996772,NaN,NaN,False,True
1991-01-31,8.269524,6.906452,NaN,6.410476,6.575714,6.644286,7.125238,7.377619,7.700000,7.970476,8.091905,NaN,9275.276000,6035.178000,NaN,NaN,False,True
1991-02-28,8.034211,6.252500,NaN,6.116316,6.194737,6.266842,6.866842,7.077368,7.472632,7.732632,7.854737,NaN,9290.982788,6060.532444,NaN,NaN,False,True
1991-03-31,8.288000,6.115161,NaN,6.093500,6.198500,6.396000,7.103000,7.353500,7.772000,8.002500,8.110000,NaN,9319.113159,6093.872506,NaN,NaN,False,True
2001-09-30,5.482941,3.069000,2.675882,2.692353,2.705882,2.824706,3.118824,3.450588,4.115294,4.512353,4.731765,5.526471,13257.826529,10627.707306,NaN,NaN,False,True
2001-10-31,5.315455,2.485161,2.267273,2.197727,2.170909,2.330455,2.725909,3.136818,3.910000,4.307273,4.566818,5.339545,13284.881000,10660.465000,NaN,NaN,False,True


In [5]:
data["RecessionFollowing18Months"] = False
data["RecessionFollowing18Months"] = data.apply(lambda x: True in data.loc[x.name:][:19]["Recession"].values, axis = 1)
data["RGDP Growth"] = np.log(data["RGDP"]).diff(12).mul(100)
data["NGDP Growth"] = np.log(data["NGDP"]).diff(12).mul(100)
data.to_csv("C:/Users/JLCat/Documents/For NDSU/EnrichmentCourses/DataScienceCertificate/STAT714/Homework/Data/ProjectYieldCurveData.csv")
data.diff(12).to_csv("C:/Users/JLCat/Documents/For NDSU/EnrichmentCourses/DataScienceCertificate/STAT714/Homework/Data/ProjectYieldCurveData12MonthDiff.csv")
